In [2]:
#from pycalphad import Model

def custom_model(model_name, contributions, basic_functions, parameter_functions, energy_functions):
    # Create the class definition as a string
    class_definition = f"class {model_name}(Model):\n"
  
    # Add contributions to the class definition
    for contribution in contributions:
        class_definition += f"    {contribution}\n"

    # Add basic functions to the class definition
    for basic_function in basic_functions:
        class_definition += f"    {basic_function}\n"
   
    # Add parameter functions to the class definition
    for method in parameter_functions:
        class_definition += f"    {method}\n"
  
    for method in energy_functions:
        class_definition += f"    {method}\n"
    # Return the class definition
    return class_definition




In [3]:
import json

def json_to_basic_functions_strings(json_input):
    function_strings = []
    template_file=open('template_functions.json')
    template_functions=json.load(template_file)
    for key in json_input:
        if key == 'none':
            break
        if key == 'defualt':
            for funs in template_functions['functions']:
                if funs['name'] in ['__new__', '__init__', '__eq__', 'moles', 'variables', 'endmember_reference_model', 'get_internal_constraints', 'build_phase']:
                    function_strings.append(funs['content']) 
        else:
            for funs in template_functions['functions']:
                if funs['name'] == key:
                    function_strings.append(funs['content']+'\n')
    return function_strings


def json_to_parameter_functions_strings(json_input):
    function_strings = []

    for key, value in json_input.items():
        if len(value) == 2:
            function_strings.append(f"def {key}(self, dbe, {', '.join(value[1])}):")
            function_strings.append(f"\t#{value[0]}")
            function_strings.append(f"\treturn {key}\n")
        else:
            function_strings.append(f"def {key}(self, dbe):")
            function_strings.append(f"\t#{value[0]}")
            function_strings.append(f"\treturn {key}\n")
    
    return function_strings

def json_to_energy_functions_strings(json_input):
    function_strings = []
    template_file=open('template_functions.json')
    template_functions=json.load(template_file)
    for key, value in json_input.items():
        if value == 'CEF-default':
            print('Need to include redlich_kister_sum function')
            for funs in template_functions['functions']:
                if funs['name'] == key:
                    function_strings.append(funs['content']+'\n')
        else:
            function_strings.append(f"def {key}(self, dbe):")
            function_strings.append(f"\t#{value}")
            function_strings.append(f"\treturn {key}\n")
    
    return function_strings


def process_model_information(jsonfile):
    j_file=open(jsonfile)
    ModelInform=json.load(j_file)
    class_name=ModelInform['model_name']
    #contribution
    energy_contributions=ModelInform['energy_contributions']
    en_cons = [f'\t("{{key}}", "{{value}}")'.format(key=key, value=value) for key, value in energy_contributions.items()]
    energy_contributions_result_string = ["contributions = [\n{}\n\t]".format(",\n".join(en_cons))]
    #basic functions
    basic_function_input=ModelInform['basic_functions']
    basic_function_strings=json_to_basic_functions_strings(basic_function_input)
    #parameter functions
    parameter_function_input=ModelInform['parameters_functions']
    parameter_function_strings=json_to_parameter_functions_strings(parameter_function_input)
    #energy functions
    energy_function_input=ModelInform['energy_functions']
    energy_function_strings=json_to_energy_functions_strings(energy_function_input)
    return class_name, energy_contributions_result_string, basic_function_strings, parameter_function_strings, energy_function_strings

In [4]:
class_name, contributions, basic_functions, parameter_functions, energy_functions = process_model_information('Model_information.json')

# Generate the class definition
dynamic_class_code = custom_model(class_name, contributions, basic_functions, parameter_functions, energy_functions)


Need to include redlich_kister_sum function
Need to include redlich_kister_sum function


In [8]:
# Print the generated class definition
print(dynamic_class_code)

# Evaluate the generated code to create the class
#exec(dynamic_class_code)

class NewModel(Model):
    contributions = [
	("ref", "reference_energy"),
	("idmix", "ideal_mixing_energy"),
	("xsmix", "excess_mixing_energy")
	]
    
    def build_phase(self, dbe):

        #Generate the symbolic form of all the contributions to this phase.

        contrib_vals = list(OrderedDict(self.__class__.contributions).values())
        if 'atomic_ordering_energy' in contrib_vals:
            if contrib_vals.index('atomic_ordering_energy') != (len(contrib_vals) - 1):
                # Check for a common mistake in custom models
                # Users that need to override this behavior should override build_phase
                raise ValueError('\'atomic_ordering_energy\' must be the final contribution')
        self.models.clear()
        for key, value in self.__class__.contributions:
            self.models[key] = S(getattr(self, value)(dbe))

    def v_i(self, dbe, i: v.Species):
    	#volume parameters
    	return v_i

    def Z(self, dbe):
    	#coordination numbers

In [9]:
file_path = "model_NewModel.py"
with open(file_path, "w") as py_file:
    f=open('template_functions.json')
    temps=json.load(f)
    py_file.write(temps['imports']+'\n')
    py_file.write(dynamic_class_code)